In [5]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from http import cookiejar
from time import sleep
import random
from lxml import html
import re
import requests_cache
from selenium import webdriver
from scrapy.selector import Selector
from time import sleep
import time
from datetime import datetime
from datetime import timedelta
import urllib
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from PIL import ImageGrab
#requests_cache.install_cache('qcyn--chuvote')
import win32api,win32con
from selenium.webdriver.common.action_chains import ActionChains

In [6]:
def get_time():
    utc_now = time.time()
    utc_time = datetime.utcfromtimestamp(utc_now)
    time2 = utc_time + timedelta(hours=8)
    #nm = str(time2.hour)+":"+str(time2.minute)
    return(time2)

In [7]:

params = {"u":"6436563121","m":"4324156859858956","cu":"6436563121","containerid":"231610_standard_cardlist_1487_yanyuan_index"}
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36",
                  'Referer': 'https://m.weibo.cn/p/231610_standard_cardlist_1487_yanyuan_index?u=6436563121&m=4324156859858956&cu=6436563121',
              "Accept":"application/json, text/plain, */*",
                "Accept-Encoding": "gzip, deflate, br",
                "Accept-Language": "en-US,en;q=0.5",
                "Host": "m.weibo.cn"}



In [ ]:
while True:
    name = []
    vote = []
    url = "https://m.weibo.cn/api/container/getIndex"
    res = requests.get(url,params = params)
    try:
        votedetail = json.loads(res.text)['data']['cards'][1]["card_group"]
        for i in votedetail[2:52]:
            name.append(i['items'][0]['title'])
            name.append(i['items'][1]['title'])
            vote.append(i['items'][0]['price2'])
            vote.append(i['items'][1]['price2'])
        a = pd.DataFrame([name,vote]).transpose()
        a[1] = a[1].apply(lambda x: x.split()[0])
        time2 =get_time()
        nm =  str(time2.day)+"_"+str(time2.hour)+"_"+str(time2.minute)
        a.to_csv("sina_niyingyuan"+nm+".csv",encoding="utf_8_sig")
        time.sleep(600)
    except:
        pass

In [ ]:
def get_vote(uid):
    params = {"u":"6436563121","m":"4324156859858956","cu":"6436563121","containerid":"231610_standard_cardlist_1487_yanyuan_index"}
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36",
                  'Referer': 'https://m.weibo.cn/p/231610_standard_cardlist_1487_yanyuan_index?u=6436563121&m=4324156859858956&cu=6436563121',
              "Accept":"application/json, text/plain, */*",
                "Accept-Encoding": "gzip, deflate, br",
                "Accept-Language": "en-US,en;q=0.5",
                "Host": "m.weibo.cn"}
    url = "https://m.weibo.cn/api/container/getIndex"
    res = requests.get(url,params = params)
    votedetail = json.loads(res.text)["data"][0]["childs"][0]["options"]
    df = pd.DataFrame(votedetail)
    df.kv = df.kv.str.replace('"',"")
    df.kv = df.kv.str.replace('{',"")
    df.kv = df.kv.str.replace('}',"")
    df.kv =df.kv.str.replace(r'[a-zA-Z]+\_*[a-zA-Z]+:',"")
    new = df.kv.str.split(",",n=7,expand = True)
    print(df.shape)
    df["aim_list"] = new[0]
    df["collectionid"] = new[1]
    df["Collection_URL"] = new[2]
    df["motto"] = new[3]
    df["order"] = new[4]
    df["peopleid"] = new[5]
    df["talents"] = new[6]
    df["video_qipuid"] = new[7]
    #df_new = df[["text","showNum"]]
    #df_new = df_new.sort_values(by=['showNum'], ascending=False)
    #df_new["order"] = range(1,100)
    return(df)

In [3]:
# private browsing
firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference("browser.privatebrowsing.autostart", True)

In [91]:
driver = webdriver.Firefox(executable_path=r'E:\geckodriver.exe',firefox_profile=firefox_profile)
driver.get("https://topstar.h5.weibo.cn/rank?rType=15&showmenu=1&topnavstyle=1&immersiveScroll=150")

In [106]:
## load all
for i in range(1,20):
    driver.execute_script("window.scrollTo(0,"+str(i*1000)+")")
    time.sleep(0.5)
driver.execute_script("window.scrollTo(0,0)")    

In [158]:
info = {}
name = []
names_ele = driver.find_elements_by_class_name("m-text-cut")

i = 0
for nm in names_ele:
    info[i] = {'name':nm.text}
    name.append(nm.text)
    i = i + 1

In [170]:
score_overall = []
score_ele = driver.find_elements_by_class_name("fm-dpb")

i = 0
for s in score_ele:
    if "." in s.text:
        score_overall.append(s.text)
        info[i].update({'score_overview':s.text})
        i = i + 1

In [ ]:
driver.execute_script("document.getElementsByClassName('m-font m-font-arrow-down')[1].click()")
time.sleep(0.5)
lt = driver.find_elements_by_class_name("open-const")[1].find_elements_by_class_name('oindex-2')[0].text
a = lt.split("\n")
info[0].update({'yuedu_rank':a[1],'yuedu_score':a[2],'hudong_rank':a[5],'hudong_score':a[6],
               'yingxiang_rank':a[9],'yingxiang_score':a[10],"aimu_rank":a[13],"aimu_score":a[14],
               "zheng_rank":a[17],"zheng_score":a[18]})

In [ ]:
#driver.execute_script("document.getElementsByClassName('m-font m-font-arrow-down')[1].click()")
driver.execute_script("document.getElementsByClassName('m-font m-font-arrow-down')[2].click()")
time.sleep(0.5)
lt = driver.find_elements_by_class_name("open-const")[1].find_elements_by_class_name('oindex-1')[0].text
a = lt.split("\n")
info[1].update({'yuedu_rank':a[1],'yuedu_score':a[2],'hudong_rank':a[5],'hudong_score':a[6],
               'yingxiang_rank':a[9],'yingxiang_score':a[10],"aimu_rank":a[13],"aimu_score":a[14],
               "zheng_rank":a[17],"zheng_score":a[18]})

In [192]:
#driver.execute_script("document.getElementsByClassName('m-font m-font-arrow-down')[2].click()")
driver.execute_script("document.getElementsByClassName('m-font m-font-arrow-down')[3].click()")
time.sleep(0.5)
lt = driver.find_elements_by_class_name("open-const")[1].find_elements_by_class_name('oindex-3')[0].text
a = lt.split("\n")
info[2].update({'yuedu_rank':a[1],'yuedu_score':a[2],'hudong_rank':a[5],'hudong_score':a[6],
               'yingxiang_rank':a[9],'yingxiang_score':a[10],"aimu_rank":a[13],"aimu_score":a[14],
               "zheng_rank":a[17],"zheng_score":a[18]})

In [196]:
driver.execute_script("document.getElementsByClassName('m-font m-font-arrow-down')[3].click()")

In [ ]:
time.sleep(0.5)

In [215]:
for i in range(4,101):
    driver.execute_script("document.getElementsByClassName('m-font m-font-arrow-down')["+str(i)+"].click()")
    time.sleep(0.5)
    lt = driver.find_elements_by_class_name("open-const")[2].text
    a = lt.split("\n")
    info[i-1].update({'yuedu_rank':a[1],'yuedu_score':a[2],'hudong_rank':a[5],'hudong_score':a[6],
               'yingxiang_rank':a[9],'yingxiang_score':a[10],"aimu_rank":a[13],"aimu_score":a[14],
               "zheng_rank":a[17],"zheng_score":a[18]})
    driver.execute_script("document.getElementsByClassName('m-font m-font-arrow-down')["+str(i)+"].click()")

In [245]:
dt = pd.DataFrame.from_dict(info,orient='index')

In [246]:
dt = dt.sort_values("score_overview",ascending=False)

In [247]:
dt.name = dt.name.apply(lambda x: x.split("-")[1])
dt.yuedu_rank = dt.yuedu_rank.apply(lambda x: x[2:])
dt.hudong_rank = dt.hudong_rank.apply(lambda x: x[2:])
dt.yingxiang_rank = dt.yingxiang_rank.apply(lambda x: x[2:])
dt.aimu_rank = dt.aimu_rank.apply(lambda x: x[2:])
dt.zheng_rank = dt.zheng_rank.apply(lambda x: x[2:])

In [251]:
dt.to_csv("sina_rank_2019_01_01.csv",encoding='utf_8_sig')

In [252]:
test

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d4c02d10-62f0-44a4-b6e3-95ed2fff5c21", element="d5b94462-3af5-452c-b150-88f2c7318043")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d4c02d10-62f0-44a4-b6e3-95ed2fff5c21", element="399165ad-82c0-461c-9172-33693163c71e")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d4c02d10-62f0-44a4-b6e3-95ed2fff5c21", element="9bf9f892-413f-48e7-9c6f-36af0d242164")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d4c02d10-62f0-44a4-b6e3-95ed2fff5c21", element="1be7e634-a2ed-4d08-b124-257ac7264c24")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d4c02d10-62f0-44a4-b6e3-95ed2fff5c21", element="20dca3a3-2c2e-434f-abb0-051d06de6432")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="d4c02d10-62f0-44a4-b6e3-95ed2fff5c21", element="20482f65-fa6d-49a1-8ae3-a2642e95b1b1")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement 

In [89]:
driver.find_elements_by_class_name("m-font m-font-arrow-down")

[]

In [88]:
sss

[]

In [53]:
test[0].execute_script("document.getElementsByClassName('m-font m-font-arrow-down')[2].click()")

TypeError: 'FirefoxWebElement' object does not support indexing

In [67]:
sel = Selector(text=driver.page_source)

In [29]:
for i in ids:
    a = i.find_elements_by_class_name('m-box-col o-tit')
    print(a.text)

AttributeError: 'list' object has no attribute 'text'

In [65]:
timetest = get_time()
record = dict()
while (timetest.day<=29):
    driver = webdriver.Firefox(executable_path=r'E:\geckodriver.exe',firefox_profile=firefox_profile)
    driver.get("https://www.iqiyi.com/v_19rqpln99w.html")
    length = '0:0'
    target = '01:20'
    while (length != target):
        sleep(random.randrange(10,20))  #load page and skip ads
        try :
            mouse = driver.find_element_by_tag_name('video')
            ActionChains(driver).move_to_element(mouse).perform()
            length = driver.find_element_by_class_name("iqp-time-dur").text
        except :
            length = '0:0'
    love = np.random.binomial(1,0.1)
    if (love==1):
        driver.execute_script("document.getElementsByClassName('like-red-heart')[0].click()")  #点爱心
    #locate:
    #first_scra = random.randint(800,1300)
    #driver.execute_script("window.scrollTo(0, 60)")
    
    time2 = get_time()
    nm = str(time2.hour)+"_"+str(time2.minute)
    hot = driver.find_element_by_class_name("basic-txt")
    record[nm] = hot.text
    print(nm+":"+hot.text)
    win32api.keybd_event(win32con.VK_SNAPSHOT, 0)
    time.sleep(0.5)
    im=ImageGrab.grabclipboard()
    im.save("E:/first-video/" + str(nm)+".png")
    
    #calculate
    mouse = driver.find_element_by_tag_name('video')
    ActionChains(driver).move_to_element(mouse).perform()
    nowtime = driver.find_element_by_class_name("iqp-time-cur")
    length = driver.find_element_by_class_name("iqp-time-dur")
    begin = datetime.strptime(nowtime.text, '%M:%S')
    end = datetime.strptime(length.text, '%M:%S')
    wait = (end-begin).total_seconds()
    #time.sleep(4.95)
    stop = np.random.binomial(1,0.1)
    if (stop==1):
        a = random.randint(5,wait)
        time.sleep(a)
        driver.execute_script("document.getElementsByTagName('video')[0].click()")
        first_scra = random.randint(800,1300)
        driver.execute_script("window.scrollTo(0, "+str(first_scra)+")")
        sleep(random.randrange(5,10))
        driver.execute_script("window.scrollTo(0,0)")
        driver.execute_script("document.getElementsByTagName('video')[0].click()")
        time.sleep(wait-a)
    else:
        time.sleep(wait)
    time.sleep(4)
    #print(length == target)
    #while (length == target):
    #    sleep(random.randrange(1,3))  #load page and skip ads
    #    print("wait")
    #    try :
    #        length = driver.find_element_by_class_name("iqp-time-dur").text
    #        print(length)
    #    except :
    #       length = '01:20'
    driver.close()




23_38:972
23_41:978
23_44:978
23_47:980
23_50:980
23_53:983
23_56:983
23_58:985
0_2:985
0_4:988
0_7:990
0_10:991
0_14:991
0_17:991
0_20:991
0_23:992
0_26:992
0_29:992
0_32:992
0_38:992
0_41:992
0_43:992
0_46:992
0_49:992
0_52:992
0_55:992
0_58:992
1_1:992
1_4:992
1_7:992
1_13:992
1_24:992
1_27:993
1_29:993
1_32:993
1_35:993
1_38:993
1_41:993
1_44:993
1_47:993
1_52:993
1_54:993
1_57:993
2_0:993
2_2:993
2_5:993
2_18:993
2_21:993
2_24:993
2_27:993
2_30:993
2_33:993
2_36:993
2_39:993
2_41:993
2_44:993
2_47:993
2_50:993
2_53:993
2_56:993
2_59:993
3_22:993
3_25:993
3_28:993
3_31:993
3_34:993
3_37:993
3_40:993
3_43:993
3_46:993
3_48:993
3_51:993
3_54:993
3_57:993
4_0:993
4_3:993
4_6:栏目简介
4_9:993
4_12:993
4_15:993
4_17:993
4_20:993
4_23:993
4_26:993
4_29:993
4_31:993
4_35:993
4_37:993
4_40:993
4_43:993
4_46:993
4_49:993
4_52:993
4_54:993
4_57:993
5_40:993
5_42:993
6_26:993
6_29:993
6_32:993
6_35:994
6_38:994
6_41:994
6_44:994
6_47:994
7_1:994
7_3:994
7_6:994
7_9:994
7_12:994
7_15:994
7_17:994


KeyboardInterrupt: 

In [6]:
https://www.iqiyi.com/v_19rqpln99w.html"
def get_time():
    utc_now = time.time()
    utc_time = datetime.utcfromtimestamp(utc_now)
    time2 = utc_time + timedelta(hours=8)
    nm = str(time2.hour)+":"+str(time2.minute)
    return(nm)

In [7]:
record = dict()

In [ ]:
i = 0
while i < 20:
    driver.get("https://www.iqiyi.com/v_19rqpgbdwc.html")
    sleep(random.randrange(15,25))
    sel = Selector(text=driver.page_source)
    hot = driver.find_element_by_class_name("basic-txt")
    nm = get_time()
    record[nm] = hot.text
    print(nm+":"+hot.text)
    sleep(1800)
    i = i + 1

15:28:418
15:58:441
23:3:613
23:33:620


In [27]:
User-Agent	
Mozilla/5.0 (Windows NT 10.0; …) Gecko/20100101 Firefox/64.0

In [42]:
utc_now = time.time()

utc_time = datetime.utcfromtimestamp(utc_now)
time2 = utc_time + timedelta(hours=8)

In [54]:
time2.now()

datetime.datetime(2018, 12, 26, 23, 18, 12, 571252)

'15_15'